In [1]:
# import Libraries
import pandas as pd
from sqlalchemy import create_engine

## Import Sales Data

In [2]:
# Create a dataframe 
df_sales = pd.read_csv('sales_data_sample.csv', encoding = 'latin1')
df_sales.head(5)


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


## Data Understanding 

In [3]:
len(df_sales['ORDERDATE'])

2823

### Extract customer related data from Sales to populate stg_customer

#### Customer Data

In [4]:
# Extract customer data from sales.csv
df_customer = df_sales [['CUSTOMERNAME', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME', 'PHONE', 'CITY',
                           'STATE', 'COUNTRY', 'TERRITORY', 'DEALSIZE' ]]
df_customer.head(5)  

,CUSTOMERNAME,CONTACTLASTNAME,CONTACTFIRSTNAME,PHONE,CITY,STATE,COUNTRY,TERRITORY,DEALSIZE
0,Land of Toys Inc.,Yu,Kwai,2125557818,NYC,NY,USA,NaN,Small
1,Reims Collectables,Henriot,Paul,26.47.1555,Reims,NaN,France,EMEA,Small
2,Lyon Souveniers,Da Cunha,Daniel,+33 1 46 62 7555,Paris,NaN,France,EMEA,Medium
3,Toys4GrownUps.com,Young,Julie,6265557265,Pasadena,CA,USA,NaN,Medium
4,Corporate Gift Ideas Co.,Brown,Julie,6505551386,San Francisco,CA,USA,NaN,Medium


In [5]:
# check number of records 
len(df_customer)

2823

#### Populate stg_Customer table

##### Create a connection to SQL Srever 

In [2]:
server = 'DB_SERVER'
database = 'DB_NAME'
#username = ''
#password = '
conn_str = f'mssql+pyodbc://@{server}/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
engine = create_engine(conn_str)
engine

Engine(mssql+pyodbc://@DB_SERVER/DB_NAME?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes)

##### Upload the Data to SQL Table

In [7]:
# Column mapping 
df_customer.columns 

Index(['CUSTOMERNAME', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME', 'PHONE', 'CITY',
       'STATE', 'COUNTRY', 'TERRITORY', 'DEALSIZE'],
      dtype='object')

In [8]:
column_mapping=  {'CUSTOMERNAME':'customer_name',
                  'CONTACTLASTNAME': 'contact_lastname',
                  'CONTACTFIRSTNAME':'contact_firstname',
                  'CITY': 'city',
                  'STATE': 'state',
                  'COUNTRY': 'country',
                  'TERRITORY': 'territory',
                  'DEALSIZE': 'deal_size',
                  'PHONE': 'phone',
                  
                 }

In [9]:
df_customer= df_customer.rename(columns = column_mapping)

In [10]:
df_customer.head(5)

,customer_name,contact_lastname,contact_firstname,phone,city,state,country,territory,deal_size
0,Land of Toys Inc.,Yu,Kwai,2125557818,NYC,NY,USA,NaN,Small
1,Reims Collectables,Henriot,Paul,26.47.1555,Reims,NaN,France,EMEA,Small
2,Lyon Souveniers,Da Cunha,Daniel,+33 1 46 62 7555,Paris,NaN,France,EMEA,Medium
3,Toys4GrownUps.com,Young,Julie,6265557265,Pasadena,CA,USA,NaN,Medium
4,Corporate Gift Ideas Co.,Brown,Julie,6505551386,San Francisco,CA,USA,NaN,Medium


In [11]:
no_records = df_customer.to_sql('stg_customer', con= engine,if_exists = 'append', index = False)
print(no_records)

27
